Installing the Julep Client

In [24]:
!pip install julep -U --quiet

In [25]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### creating julep client with the api key


In [26]:
from julep import Client
import os

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [27]:
# Defining the agent
name = "Jacob"
about = "A travel assistant that helps plan the perfect trip."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [45]:
import yaml

openweathermap_api_key = "YOUR_API_KEY"
brave_api_key = "YOUR_API_KEY"

# Defining the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/src/schemas/create_task_request.json
name: Julep Tourist Plan With Weather And Attractions
description: A task that plans a trip with weather and attractions.

########################################################
####################### INPUT SCHEMA ##################
########################################################                       
input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
      description: The locations to search for.

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

- name: weather
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: {openweathermap_api_key}

- name: internet_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: {brave_api_key}

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Fetch weather data for each location
- over: $ steps[0].input.locations
  map:
    tool: weather
    arguments:
      location: $ _

# Step 1: Search Wikipedia for tourist attractions for each location
- over: $ steps[0].input.locations
  map:
    tool: internet_search
    arguments:
      query: $ 'tourist attractions in ' + _

# Step 2: Zip locations, weather, and attractions into a list of tuples [(location, weather, attractions)]
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.locations,
          [output['result'] for output in steps[0].output],
          steps[1].output
        )
      )

# Step 3: Create an itinerary for each location
- over: $ _['zipped']
  parallelism: 3
  # Inside the map step, each `_` represents the current element in the list
  # which is a tuple of (location, weather, attractions)
  map:
    prompt:
    - role: system
      content: >-
        $ f'''You are {{agent.name}}. Your task is to create a detailed itinerary
        for visiting tourist attractions in some locations.
        The user will give you the following information for each location:

        - The location
        - The current weather condition
        - The top tourist attractions'''
    - role: user
      content: >-
        $ f'''Location: "{{_[0]}}"
        Weather: "{{_[1]}}"
        Attractions: "{{_[2]}}"'''
    unwrap: true

# Step 4: Create a final plan by joining the activities for each location
- evaluate:
    final_plan: |-
      $ '\\n---------------\\n'.join(activity for activity in _)
""")

In [46]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [47]:
execution = client.executions.create(
    task_id=task.id,
    input={
         "locations": ["New York", "London", "Paris", "Tokyo", "Sydney"]
    }
)

print("Started an execution. Execution ID:", execution.id)

Started an execution. Execution ID: 067ffa47-3b83-79b7-8000-eaaa1a013f0e


## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [48]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

if 'final_plan' in execution.output:
    print(execution.output['final_plan'])
else:
    print(execution.output)


Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  succeeded
--------------------------------------------------
Here's a detailed itinerary for exploring New York, tailored to the current weather and top attractions:

### Day 1: Arrival and Exploration

**Morning:**
- **Central Park:**
  - Begin your day with a brisk walk through Central Park. It's a perfect way to enjoy the morning with temperatures arou

In [49]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'locations': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']}
--------------------------------------------------
Transition type:  init_branch
Transition output:  Sydney
--------------------------------------------------
Transition type:  init_branch
Transition output:  London
--------------------------------------------------
Transition type:  init_branch
Transition output:  Tokyo
--------------------------------------------------
Transition type:  init_branch
Transition output:  Paris
--------------------------------------------------
Transition type:  init_branch
Transition output:  New York
--------------------------------------------------
Transition type:  finish_branch
Transition output:  {'result': 'In London, the current weather is as follows:\nDetailed status: scattered clouds\nWind speed: 8.05 m/s, direction: 222°\nHumidity: 45%\nTemperature: \n  - Current: 14.87°C\n  - High: 15.83°C\n  - Low: 13.86°C\n  - Feels like: 13.59°C\nR

## Related Concepts

- [Agents](https://docs.julep.ai/concepts/agents)
- [Tasks](https://docs.julep.ai/concepts/tasks)
- [Tasks](https://docs.julep.ai/concepts/tasks)
